In [212]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from datasets import load_dataset
import re
from tqdm import tqdm
import torch
from torch.nn import functional as F
from torch.optim import AdamW
import matplotlib.pyplot as plt
from torch import nn

device='cuda'
ds = load_dataset("rajpurkar/squad")

In [213]:
model=AutoModelForCausalLM.from_pretrained('../model/normal_model')
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B")
tokenizer.pad_token=tokenizer.eos_token
tokenizer.pad_token_id = tokenizer.eos_token_id

In [214]:
data_size = 100
data_size_v = 100
size = int(data_size/4)
size_v = int(data_size_v/4)
train_dataset=ds["train"].shuffle(seed=42).select(range(100))
validation_dataset=ds["validation"].shuffle(seed=42)

In [215]:
def reshape(dataset):
    reshape_dataset = [0] * len(dataset)
    for i in range(len(dataset)):
        reshape_dataset[i]="C: "+dataset[i]["context"]+" Q: "+dataset[i]["question"]+" A: "+dataset[i]["answers"]["text"][0]
    reshape_dataset = [item for item in reshape_dataset if item != '' and len(item) >= 50 and '@' not in item]
    reshape_dataset = [re.sub(r'[^a-zA-Z0-9 .:?]', '', item) for item in reshape_dataset]
    reshape_dataset = [re.sub(r'\s+', ' ', item) for item in reshape_dataset]
    return reshape_dataset[:data_size]

In [216]:
def make_data(data):
    dataset=reshape(data)
    data = []
    for text in tqdm(dataset, desc="Tokenizing dataset"):
        cq_len=len(tokenizer(text[:text.find("A:")])['input_ids'])
        tokenized = tokenizer(text, padding="max_length", max_length=512, truncation=True, return_tensors="pt")
        input_ids = tokenized['input_ids'].squeeze().tolist()
        attention_mask = tokenized['attention_mask'].squeeze().tolist()
        labels = input_ids[1:] + [tokenizer.pad_token_id]
        for i in range(min(cq_len-2, 512)):
            labels[i]=128001
        data.append({"input_ids": input_ids, "labels": labels, "attention_mask":attention_mask})
    
    return data

In [217]:
data = make_data(train_dataset)
data_v = make_data(validation_dataset)

Tokenizing dataset:   0%|          | 0/100 [00:00<?, ?it/s]

Tokenizing dataset: 100%|██████████| 100/100 [00:00<00:00, 1338.46it/s]


In [218]:
labelsdata=[]
for i in range(data_size):
    data_l = [x for x in data[i]['labels'] if x != 128001]
    labelsdata.append(data_l)


In [223]:
for date_la in labelsdata:
    print(date_la) 
    print(tokenizer.decode(date_la))

[362, 25, 220, 5833]
 A: 84
[362, 25, 6603]
 A: books
[362, 25, 279, 11145]
 A: the executive
[362, 25, 1556, 7910, 299]
 A: Anjiro
[362, 25, 30853]
 A: loops
[362, 25, 220, 17, 13, 17, 7239]
 A: 2.2 billion
[362, 25, 28058, 24245, 315, 279, 549, 815, 13, 99452, 22967]
 A: Military Governor of the U.S. Occupation Zone
[362, 25, 279, 14198, 3026]
 A: the brown men
[362, 25, 5070, 1436, 387, 17550, 311, 279, 10977, 520, 12474, 5326]
 A: resources could be targeted to the communities at greatest risk
[362, 25, 26828, 61495]
 A: honey ants
[362, 25, 578, 356, 3746, 7977]
 A: The Cossacks
[362, 25, 26742, 5346, 285]
 A: verdigris
[362, 25, 220, 4468, 15, 82]
 A: 1970s
[362, 25, 8219, 55551, 329, 647]
 A: Sun Jiadong
[362, 25, 4783, 11060]
 A: House Master
[362, 25, 11888]
 A: nine
[362, 25, 384, 14946, 324, 598, 323, 47715, 1371, 51835]
 A: echiurans and sipunculan
[362, 25, 44193]
 A: Religion
[362, 25, 1370, 5893, 32893, 24569]
 A: paralyzes muscles
[362, 25, 8305, 24520]
 A: tituli
[362,

In [221]:
datav=[0,0,0,0,0]
datav2 = datav[:0]

In [222]:
datav2

[]

In [2]:
from datasets import load_dataset
from transformers import AutoModelForCausalLM, Trainer, TrainingArguments, AutoTokenizer
import re
from tqdm import tqdm
import torch
from torch.nn import functional as F
from torch.optim import AdamW
import matplotlib.pyplot as plt
from torch import nn



def reshape(dataset):
    reshape_dataset = [0] * len(dataset)
    for i in range(len(dataset)):
        reshape_dataset[i]="C: "+dataset[i]["context"]+" Q: "+dataset[i]["question"]+" A: "+dataset[i]["answers"]["text"][0]
    reshape_dataset = [item for item in reshape_dataset if item != '' and len(item) >= 50 and '@' not in item]
    reshape_dataset = [re.sub(r'[^a-zA-Z0-9 .:?]', '', item) for item in reshape_dataset]
    reshape_dataset = [re.sub(r'\s+', ' ', item) for item in reshape_dataset]
    return reshape_dataset[:data_size]

def max_length(dataset):
    max_eval=0
    for i in dataset:
        max_eval = len(i) if len(i) > max_eval else max_eval
    print(max_eval)
    return

def batch(input, size):
    batch_train=[]
    for i in range(size):
        batch_input=[input[4*i+0], input[4*i+1], input[4*i+2], input[4*i+3]]
        batch_train.append(batch_input)

    return batch_train

def make_data(data):
    dataset=reshape(data)
    data = []
    for text in tqdm(dataset, desc="Tokenizing dataset"):
        if len(tokenizer(text)['input_ids']) <= 256:
            cq_len=len(tokenizer(text[:text.find("A:")])['input_ids'])
            tokenized = tokenizer(text, padding="max_length", max_length=256, truncation=True, return_tensors="pt")
            input_ids = tokenized['input_ids'].squeeze().tolist()
            attention_mask = tokenized['attention_mask'].squeeze().tolist()
            labels = input_ids[1:] + [tokenizer.pad_token_id]
            for i in range(min(cq_len-2, 256)):
                labels[i]=128001
            data.append({"input_ids": input_ids, "labels": labels, "attention_mask":attention_mask})
    
    return data

def make_tensor(data, type, size):
    tmp = [item[type] for item in data]
    tmp = batch(tmp, size)
    tensor=torch.tensor(tmp, dtype=torch.long)
    return tensor

ds = load_dataset("rajpurkar/squad")
device='cuda'
student_model = AutoModelForCausalLM.from_pretrained("../model/LoRA_distill_model")
teacher_model = AutoModelForCausalLM.from_pretrained("./model/teacher_model")
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B")
tokenizer.pad_token = tokenizer.eos_token
tokenizer.pad_token_id = tokenizer.eos_token_id

for name, param in student_model.named_parameters():
    if not 'base' in name:
        param.requires_grad = True
        # if 'self' in name:
        #     param.requires_grad = True

data_size = 100

train_dataset=ds["train"].shuffle(seed=42)

data = make_data(train_dataset)
size=int(len(data)/4)
input_ids_tensor = make_tensor(data, "input_ids", size)
labels_tensor = make_tensor(data, "labels", size)
attention_mask_tensor = make_tensor(data, "attention_mask", size)


vocab_size = student_model.config.vocab_size
criterion = torch.nn.CrossEntropyLoss(ignore_index=128001)

criterion.to(device)
input_ids_tensor=input_ids_tensor.to(device)
labels_tensor=labels_tensor.to(device)
attention_mask_tensor=attention_mask_tensor.to(device)

student_model.to(device)
teacher_model.to(device)

epochs = 3
lr=5e-5
temperature = 1

student_model.train()
teacher_model.eval()

for name, param in student_model.named_parameters():
    print(f"{name}: requires_grad={param.requires_grad}")

Tokenizing dataset: 100%|██████████| 100/100 [00:00<00:00, 952.34it/s]


model.embed_tokens.weight: requires_grad=True
model.layers.0.self_attn.q_proj.base_layer.weight: requires_grad=False
model.layers.0.self_attn.q_proj.lora_A.default.weight: requires_grad=True
model.layers.0.self_attn.q_proj.lora_B.default.weight: requires_grad=True
model.layers.0.self_attn.k_proj.base_layer.weight: requires_grad=False
model.layers.0.self_attn.k_proj.lora_A.default.weight: requires_grad=True
model.layers.0.self_attn.k_proj.lora_B.default.weight: requires_grad=True
model.layers.0.self_attn.v_proj.base_layer.weight: requires_grad=False
model.layers.0.self_attn.v_proj.lora_A.default.weight: requires_grad=True
model.layers.0.self_attn.v_proj.lora_B.default.weight: requires_grad=True
model.layers.0.self_attn.o_proj.weight: requires_grad=True
model.layers.0.mlp.gate_proj.weight: requires_grad=True
model.layers.0.mlp.up_proj.weight: requires_grad=True
model.layers.0.mlp.down_proj.weight: requires_grad=True
model.layers.0.input_layernorm.weight: requires_grad=True
model.layers.0

In [4]:
print(student_model.named_parameters())

<generator object Module.named_parameters at 0x7f7c317125f0>


In [26]:
import torch
import torch.nn.functional as F
P = torch.Tensor([0.4, 0.4, 0.2])
Q = torch.Tensor([0.333, 0.333, 0.333])

student_prob=F.log_softmax(P, dim=-1)
teacher_prob=F.softmax(Q, dim=-1)
print(student_prob)
kldiv_loss=F.kl_div(student_prob, teacher_prob, reduction="none")
print(kldiv_loss)
kldiv_loss.sum()

tensor([-1.0363, -1.0363, -1.2363])
tensor([-0.0208, -0.0208,  0.0459])


tensor(0.0043)

In [12]:
kldiv_loss

tensor([-0.0060, -0.0460,  0.0606])

tensor([-0.0060, -0.0460,  0.0606])

In [14]:
import torch
import torch.nn.functional as F

P = torch.Tensor([0.36, 0.48, 0.16])
Q = torch.Tensor([0.333, 0.333, 0.333])

# Pは確率分布としてそのまま使用し、Qに対してlogを取る
teacher_prob = F.softmax(Q, dim=-1)

# 温度スケーリングを適用
temperature = 1
student_prob = P / temperature  # Pはすでに確率分布
student_prob = torch.log(student_prob)  # Pの対数を取る

# KLダイバージェンスの計算
kldiv_loss = F.kl_div(student_prob, teacher_prob, reduction="none")

print(kldiv_loss.sum())

tensor(0.0975)


In [1]:
from datasets import load_dataset
from transformers import AutoModelForCausalLM, Trainer, TrainingArguments, AutoTokenizer
import optuna
import re
from tqdm import tqdm
import torch
from torch.nn import functional as F
from torch.optim import AdamW
import matplotlib.pyplot as plt
from torch import nn
import gc

ds = load_dataset("rajpurkar/squad")
device='cuda'
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B")
tokenizer.pad_token = tokenizer.eos_token
tokenizer.pad_token_id = tokenizer.eos_token_id

student_model = AutoModelForCausalLM.from_pretrained("../Distill/model/distill_model")
teacher_model = AutoModelForCausalLM.from_pretrained("./model/teacher_model")

data_size = 100
data_size_v = 600

torch.set_printoptions(profile="full")

def reshape(dataset, d_size):
    reshape_dataset = [0] * len(dataset)
    for i in range(len(dataset)):
        reshape_dataset[i]="C: "+dataset[i]["context"]+" Q: "+dataset[i]["question"]+" A: "+dataset[i]["answers"]["text"][0]
    reshape_dataset = [item for item in reshape_dataset if item != '' and len(item) >= 50 and '@' not in item]
    reshape_dataset = [re.sub(r'[^a-zA-Z0-9 .:?]', '', item) for item in reshape_dataset]
    reshape_dataset = [re.sub(r'\s+', ' ', item) for item in reshape_dataset]
    return reshape_dataset[:d_size]

def max_length(dataset):
    max_eval=0
    for i in dataset:
        max_eval = len(i) if len(i) > max_eval else max_eval
    print(max_eval)
    return

def batch(input, size):
    batch_train=[]
    for i in range(size):
        batch_input=[input[4*i+0], input[4*i+1], input[4*i+2], input[4*i+3]]
        batch_train.append(batch_input)

    return batch_train

def make_data(data, d_size):
    dataset=reshape(data, d_size)
    data = []
    for text in tqdm(dataset, desc="Tokenizing dataset"):
        if len(tokenizer(text)['input_ids']) <= 256:
            cq_len=len(tokenizer(text[:text.find("A:")])['input_ids'])
            tokenized = tokenizer(text, padding="max_length", max_length=256, truncation=True, return_tensors="pt")
            input_ids = tokenized['input_ids'].squeeze().tolist()
            attention_mask = tokenized['attention_mask'].squeeze().tolist()
            labels = input_ids[1:] + [tokenizer.pad_token_id]
            for i in range(min(cq_len-2, 256)):
                labels[i]=128001
            data.append({"input_ids": input_ids, "labels": labels, "attention_mask":attention_mask})
    
    return data

def make_tensor(data, type, size):
    tmp = [item[type] for item in data]
    tmp = batch(tmp, size)
    tensor=torch.tensor(tmp, dtype=torch.long)
    return tensor

def objective(trial):
    lr = trial.suggest_float("lr", 1e-7, 1e-4, log=True)
    temperature = trial.suggest_float("temperature", 1.0, 10.0)

    student_model=AutoModelForCausalLM.from_pretrained("./model/optim_distilledmodel")
    student_model.to(device)
    student_model.train()
    optimizer = AdamW(student_model.parameters(), lr=lr)
    losses = 0
    for i in tqdm(range(size_v)):
        optimizer.zero_grad()

        student_logits = student_model(input_ids=input_ids_tensor_v[i], attention_mask=attention_mask_tensor_v[i]).logits.view(-1, vocab_size)

        with torch.no_grad():
            teacher_logits = teacher_model(input_ids=input_ids_tensor_v[i], attention_mask=attention_mask_tensor_v[i]).logits.view(-1, vocab_size)
        mask = labels_tensor[i].view(-1) != 128001
        student_prob = F.log_softmax(student_logits[mask] / temperature, dim=-1)
        teacher_prob = F.softmax(teacher_logits[mask] / temperature, dim=-1)

        kl_loss = F.kl_div(student_prob, teacher_prob, reduction="none").sum(dim=-1).sum()
        kl_loss.backward()
        optimizer.step()
        losses += kl_loss
        
    del student_logits, teacher_logits, student_prob, teacher_prob
    return losses

train_dataset=ds["train"].shuffle(seed=42)
validation_dataset = ds["validation"].shuffle(seed=42)

data = make_data(train_dataset, data_size)
data_v = make_data(validation_dataset, data_size_v)
size=int(len(data)/4)
size_v=100

input_ids_tensor = make_tensor(data, "input_ids", size)
labels_tensor = make_tensor(data, "labels", size)
attention_mask_tensor = make_tensor(data, "attention_mask", size)
input_ids_tensor_v = make_tensor(data_v, "input_ids", size_v)
labels_tensor_v = make_tensor(data_v, "labels", size_v)
attention_mask_tensor_v = make_tensor(data_v, "attention_mask", size_v)

vocab_size = teacher_model.config.vocab_size
criterion = torch.nn.CrossEntropyLoss(ignore_index=128001)

criterion.to(device)
input_ids_tensor=input_ids_tensor.to(device)
labels_tensor=labels_tensor.to(device)
attention_mask_tensor=attention_mask_tensor.to(device)
input_ids_tensor_v=input_ids_tensor_v.to(device)
labels_tensor_v=labels_tensor_v.to(device)
attention_mask_tensor_v=attention_mask_tensor_v.to(device)


student_model.to(device)
student_model.eval()

student_logits = student_model(input_ids=input_ids_tensor[0], attention_mask=attention_mask_tensor[0]).logits.view(-1, vocab_size)
        

Tokenizing dataset: 100%|██████████| 600/600 [00:00<00:00, 1132.94it/s]


In [3]:
student_logits.view(-1, vocab_size).size()

torch.Size([1024, 128256])